In [165]:
import numpy as np
from matplotlib import pyplot as plt
from keras.datasets import mnist
from random import sample

(x_train, y_train), (x_test, y_test) = mnist.load_data()
SCALE_FACTOR = 255
WIDTH = x_train.shape[1]
HEIGHT = x_train.shape[2]
x_train = x_train.reshape(x_train.shape[0],WIDTH*HEIGHT).T / SCALE_FACTOR
x_test = x_test.reshape(x_test.shape[0],WIDTH*HEIGHT).T  / SCALE_FACTOR

# Mnist from scratch

* simple 2 layer MLP
* 1 hidden layer size 10, reLU activation
* 1 output layer, size 10, softmax activation
* based on tutorial at https://www.youtube.com/watch?v=w8yWXqWQYmU

In [166]:
# global run settings
ETA = 0.01
EPOCHS=1000
#initialise input
A_0 = np.asarray([np.reshape(np.asarray(x),[784,]) for x in x_train.T]).T
print(A_0.shape)
#initalise weights
W_1 = np.random.randn(10,784) * np.sqrt(1./(784))
b_1 = np.random.randn(10,1) * np.sqrt(1./(10))
W_2 = np.random.randn(10,10) * np.sqrt(1./(20))
b_2 = np.random.randn(10,1) * np.sqrt(1./(784))
np.mean(W_1) #should be ~~0

(784, 60000)


-0.00020535999774438235

In [174]:
def reLU(x):
    return np.maximum(x,0)

def softmax(Z):
    """Compute softmax values for each sets of scores in x."""
    exp = np.exp(Z - np.max(Z))
    return exp / exp.sum(axis=0)

def reLU_diff(x):
    return x > 0

# forward propagate
def forward_prop(A_0, W_1, b_1, W_2, b_2):
    z_1 = W_1.dot(A_0) + b_1
    # relu
    A_1 = reLU(z_1)
    z_2 = W_2.dot(A_1) + b_2
    # apply softmax
    A_2 = softmax(z_2)
    return z_1, A_1, z_2, A_2

def one_hot_Y(classnum):
    res = np.zeros(10)
    res[classnum] = 1
    return res.T

def back_prop(A_0,A_1,A_2,z_1,Y,W_1,b_1,W_2,b_2):
    samp_num = Y.shape[1] #number of training samples
    # ~~~~~~~~~ dC/dW_2 ~~~~~~~~~~~~
    dz_2 = 2*(A_2 - Y)
    dW_2 = 1/(samp_num) * dz_2.dot(A_1.T)
    # ~~~~~~~~~ dC/db_2 ~~~~~~~~~~~~
    db_2 = 1/(samp_num) * np.sum(dz_2,1).reshape([10,1])
    # ~~~~~~~~~ dC/dW_1 ~~~~~~~~~~~~
    dz_1 = W_2.T.dot(dz_2) * reLU_diff(z_1) 
    dW_1 = 1/(samp_num) * dz_1.dot(A_0.T)
    # ~~~~~~~~~ dC/db_1 ~~~~~~~~~~~~
    db_1 = 1/(samp_num) * np.sum(dz_1,1).reshape([10,1])
    W_1 -= ETA * dW_1
    W_2 -= ETA * dW_2
    b_1 -= ETA * db_1
    b_2 -= ETA * db_2
    return W_1, b_1, W_2, b_2

def return_acc(A_2, Y):
    #want to iterate over cols
    A_2 = A_2.T
    Y = Y.T
    correct = 0
    for i in range(len(A_2)):
        if np.argmax(A_2[i])==np.argmax(Y[i]):
            correct += 1
    return (correct / len(A_2))

# let's perform gradient descent
Y = np.asarray([one_hot_Y(y) for y in y_train]).T
for i in range(EPOCHS):
    z_1, A_1, z_2, A_2 = forward_prop(A_0, W_1, b_1, W_2, b_2)
    W_1, b_1, W_2, b_2 = back_prop(A_0, A_1, A_2, z_1, 
                                    Y, W_1, b_1, W_2, b_2)
    acc = return_acc(A_2,Y)
    print("Training accuracy: ",acc)

Training accuracy:  0.7541
Training accuracy:  0.7544
Training accuracy:  0.7549833333333333
Training accuracy:  0.75555


KeyboardInterrupt: ignored

In [173]:
# lets do some testing
input = np.asarray([np.reshape(np.asarray(x),[784,]) for x in x_test.T]).T
labels = np.asarray([one_hot_Y(y) for y in y_test]).T
_,_,_,preds = forward_prop(input,W_1,b_1,W_2,b_2)
acc = return_acc(preds,labels)
print("Accuracy: ",acc)

Accuracy:  0.7588


Nice.